In [1]:
from window_generator import WindowGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
from read_data import *
from methods import *
MAX_EPOCHS = 20
VOICE = 2

In [2]:
def compile_and_fit(model, window, patience=2, verbose=0):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min',
                                                     restore_best_weights=True)
#  Early stopping when there is no improvement in loss. 
    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanAbsoluteError()])

# At the end of each epoch, the model will iterate over the validation dataset and compute the validation loss and validation metrics.
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping],
                        verbose=verbose)
    return history

In [3]:
multi_step_window = WindowGenerator(input_width=54, label_width=1, shift=1, batch_size=1, voice_number=VOICE)

for example_inputs, example_labels in multi_step_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

print(multi_step_window.train)
# print(len(multi_step_window.label_columns_indices))

Inputs shape (batch, time, features): (1, 54, 6)
Labels shape (batch, time, features): (1, 1, 23)
<TakeDataset shapes: ((None, 54, 6), (None, 1, 23)), types: (tf.float32, tf.float32)>


In [4]:
n_output_nodes = len(multi_step_window.label_columns_indices)

initializer = tf.keras.initializers.RandomUniform(minval=-0.50, maxval=0.50)
linear_multi_step = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, kernel_initializer=initializer, activation='relu'),
    tf.keras.layers.Dense(units=64, kernel_initializer=initializer, activation='relu'),
    tf.keras.layers.Dense(units=n_output_nodes, activation='softmax'),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', linear_multi_step(multi_step_window.example[0]).shape)

# Train the model
history = compile_and_fit(linear_multi_step, multi_step_window)

print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
print(
    "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        len(history.history['loss']),
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

# Evaluate the model
print("Evaluate")
result = linear_multi_step.evaluate(multi_step_window.test)
dict(zip(linear_multi_step.metrics_names, result))

Input shape: (1, 54, 6)
Output shape: (1, 1, 23)
Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights 
epochs: 17, loss: 0.0086, mean_absolute_error: 0.0120, val_loss: 0.0084, val_mean_absolute_error: 0.0117
Evaluate
376/376 [==============================] - 0s 876us/step - loss: 0.0071 - mean_absolute_error: 0.0106


{'loss': 0.007111494895070791, 'mean_absolute_error': 0.010592873208224773}

In [5]:
n_output_nodes = len(multi_step_window.label_columns_indices)

initializer = tf.keras.initializers.RandomUniform(minval=-0.50, maxval=0.50)
rnn = tf.keras.Sequential([
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(units=n_output_nodes, activation='softmax'),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', rnn(multi_step_window.example[0]).shape)

# Train the model
history = compile_and_fit(rnn, multi_step_window)

print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
print(
    "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        len(history.history['loss']),
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

# Evaluate the model
print("Evaluate")
result = rnn.evaluate(multi_step_window.test)
dict(zip(rnn.metrics_names, result))

Input shape: (1, 54, 6)
Output shape: (1, 1, 23)
Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights 
epochs: 15, loss: 0.0085, mean_absolute_error: 0.0178, val_loss: 0.0098, val_mean_absolute_error: 0.0190
Evaluate
376/376 [==============================] - 1s 3ms/step - loss: 0.0092 - mean_absolute_error: 0.0193


{'loss': 0.00924970954656601, 'mean_absolute_error': 0.01932719722390175}

In [6]:
# Get last window
def predict_new_pitches(model, n=100):
    last_window = None
    last_pitch = None
    last_duration = None
    all_predicted_pitches = []
    all_probs = []

    for i in range(n):
        if last_window is None:
            # Use an iterator to get the last window of the dataset. 
            for last_window, labels in multi_step_window.test_no_shuffle.as_numpy_iterator():
                pass

#             print('window:', type(last_window),last_window ,'\n====\nlabels', labels)
#             print('window shape:', last_window.shape, ' label shape:', labels.shape)
        
        # Model is expecting a shape of (batch, timestep, features) but for one sample this is (6,6)
        # Add dimension to input
        probabilities = model.predict(np.array([last_window]))[0]
        probabilities = probabilities[0]
        all_probs.append(probabilities)
        
        predicted_pitch = get_pitch_from_probability(
            probabilities,
            key=multi_step_window.pitch_conversion_key,
            method=SelectionMethod.PROB, n=10,
        )
        all_predicted_pitches.append(predicted_pitch)
        features = get_pitch_features(predicted_pitch)

        if last_pitch is None:
            last_pitch = get_voice(VOICE)[-1]

        # - find new duration based on previous and current pitch
        if last_pitch == predicted_pitch:
            if last_duration is None:
                duration = multi_step_window.df.iloc[-1]["dur"] + 1   # bug .
            else:
                duration = last_duration + 1
        else:
            duration = 1

        last_duration = duration
        last_pitch = predicted_pitch

        # create new dataframe
        new_df = pd.DataFrame(data={"dur": duration}, dtype=float, index=[0])
        (
            new_df["log_pitch"],
            new_df["chroma_x"],
            new_df["chroma_y"],
            new_df["c5_x"],
            new_df["c5_y"],
        ) = features
        # - Normalise features
        new_df = (new_df - multi_step_window.mean_df) / multi_step_window.std_df
        # - Make new window based on previous timesteps with a slide of 1 and the normalised features
        last_window = np.append(last_window[1:], new_df.values.tolist(), axis=0)
    return all_predicted_pitches, all_probs

In [8]:
predictions, probs = predict_new_pitches(rnn, n=300)
print(predictions)
print(probs[-1])
sum(probs[-1])
# wg = WindowGenerator(input_width=6, label_width=1, shift=1, batch_size=1)
# print(f'train {sum(1 for _ in wg.train)} val {sum(1 for _ in wg.val)} test {sum(1 for _ in wg.test)} ')
# print(f'full dataset {sum(1 for _ in wg.full_dataset)}')
# print(f'data size {wg.df.shape[0]} | file voice size: {len(get_voice(0))}')

# max(get_voice(2))



[54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 52, 52, 52, 52, 52, 52, 40, 52, 52, 52, 52, 52, 52, 52, 54, 54, 52, 52, 52, 52, 52, 52, 51, 51, 52, 52, 51, 51, 49, 49, 49, 49, 49, 49, 49, 49, 47, 47, 47, 47, 47, 59, 47, 47, 47, 47, 59, 47, 47, 59, 47, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 47, 47, 49, 49, 52, 54, 54, 52, 52, 52, 52, 52, 52, 62, 50, 50, 50, 50, 48, 49, 49, 49, 49, 49, 59, 47, 47, 47, 47, 47, 47, 47, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 59, 59, 52, 52, 52, 52, 52, 52, 50, 50, 51, 51, 49, 49, 49, 49, 52, 50, 50, 50, 50, 49, 49, 47, 47, 52, 52, 52, 52, 52, 54, 54, 52, 52, 52, 52, 52, 52, 51, 51, 59, 47, 59, 59, 59, 59, 52, 52, 50, 50, 49, 49, 47, 47, 40, 40, 40, 52, 52, 52, 50, 50, 50, 50, 49, 49, 47, 47, 52, 52, 52, 52, 52, 52, 54, 54, 54, 54, 52, 52, 52, 52, 52, 52, 50, 50, 50, 50, 49, 49, 42, 54, 54, 54, 57, 57, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 46, 46, 56, 56, 59, 59, 57, 57, 57, 57, 57, 57, 57, 57, 59, 59, 57, 57, 57,

0.001100204305458874